In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dense, BatchNormalization, ReLU, Dropout, Conv2D, MaxPooling2D, Flatten, Add#, Average, Concatenate, SpatialDropout2D
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras import activations
# from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import imgaug.augmenters as iaa

# from google.colab import drive
# from google.colab import files
# import pdb; 
# # pdb.set_trace()

In [ ]:
# drive.mount("/content/drive", force_remount=True)
# %cd drive/MyDrive/Colab Notebooks/
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("Invalid device or cannot modify virtual devices once initialized.")
  pass
physical_devices[0]

# **Load Data & PreProcessing ==================================**

In [ ]:
label_encoder = {"Basketball" : 0, "Football" : 1, "Rowing" : 2, "Swimming" : 3, "Tennis" : 4, "Yoga" : 5}

***Train data***

In [ ]:
augmentation = iaa.Sequential([
    # 1. Flip
    iaa.Fliplr(0.5),
    # 2. Affine
    iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
               rotate=(-10, 10),
               scale=(0.9, 1.2)),
               
    # 3. Multiply
    iaa.Multiply((0.8, 1.2)),
    # 4. Linearcontrast
    iaa.LinearContrast((0.8, 1.2)),
    # Perform methods below only sometimes
    iaa.Sometimes(0.7,
        # 5. GaussianBlur
        iaa.GaussianBlur((0.2, 0.5))
        )
])


In [25]:
path = f".\\DataSet\\Train\\"
imgsP1 = os.listdir(path)
TrainImgs = []
TrainLabel = []
# x = 325 , y = 420
# n=0
for f in imgsP1: 
  img = cv2.imread(path+f"\\"+f)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)
  img = cv2.resize(img, (192,192), interpolation = cv2.INTER_AREA)
  
  # print(n)
  # n+=1

  TrainImgs.append(img/255)
  TrainLabel.append(label_encoder[f.split('_')[0]])
  for i in range(3):
    augmented_img = augmentation(image=img)
    augmented_img /= 255
    TrainImgs.append(augmented_img)
    TrainLabel.append(label_encoder[f.split('_')[0]])
  # pdb.set_trace()

TrainImgs = np.array(TrainImgs)
TrainLabel = np.reshape(TrainLabel,(-1,1))
del imgsP1

In [ ]:
TrainLabel.shape,"=========",TrainImgs.shape

In [ ]:
[plt.imshow(TrainImgs[i,:])for i in range(15)]

In [ ]:
np.save("TrainImgs.npy",TrainImgs)

In [ ]:
np.save("TrainLabel.npy",TrainLabel)

***Test data***

In [ ]:
path2 = f".\\DataSet\\test_\\"
imgsP2 = os.listdir(path2)
TestImgs = []
testID = []
# x = 500 , y = 700
for f in imgsP2:
  img = cv2.imread(path2+f"\\"+f)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)
  
  # m = img.mean(axis=(0,1,2))
  # s = img.std(axis=(0,1,2))
  # img -= m
  # img /= s
  img /= 255

  img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
  TestImgs.append(img)
  testID.append(f)

TestImgs = np.array(TestImgs)
del imgsP2

In [ ]:
len(testID),"=========",TestImgs.shape

In [ ]:
TrainImgs = np.load(".\\TrainImgs.npy")
TrainLabel= np.load(".\\TrainLabel.npy")

***train test split***

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(TrainImgs, TrainLabel, test_size=0.2, random_state=32)

In [ ]:
X_train.shape,"===",y_train.shape

In [ ]:
del TrainImgs
del TrainLabel

# **Model1 ==================================**

In [ ]:
def createModel (inputSize = (128,128,3)):
  x = tf.keras.Input(shape=(inputSize))
  # x_ = data_augmentation(x)
# convolutional layers
  #  1st block
  convL1_1 = Conv2D( 16, 5, activation='relu', padding="same")(x)
  convL1_2 = Conv2D( 32, 5, activation='relu', padding="same")(convL1_1)
  poolL1 = MaxPooling2D(pool_size=8 ,strides=8, padding='valid')(convL1_2)
  bnL1 = BatchNormalization()(poolL1)


  #  2nd block
  convL2_1 = Conv2D( 64, 3, activation='relu', padding="same")(bnL1)
  convL2_2 = Conv2D( 128, 3, activation='relu', padding="same")(convL2_1)
      # skip connection
  # skipConn2 = Add()([reluL2_2, poolL1])
  poolL2 = MaxPooling2D(pool_size=4, strides=4, padding='valid')(convL2_2)
  bnL2 = BatchNormalization()(poolL2)

# Flattened
  flattend = Flatten()(bnL2)

# fully connected layer tfl.Activation("tanh")
  #  layer1
  fc1 = Dense(256, activation='linear', use_bias=True, kernel_regularizer=L1L2(l1=0.5, l2=2),)(flattend)
  bn1 = BatchNormalization()(fc1)
  relu1 = ReLU()(bn1)
  d1 = Dropout(0.25)(relu1)
  #  layer2
  fc2 = Dense(128, activation='linear', use_bias=True, kernel_regularizer=L1L2(l1=0.2, l2=1))(d1)
  bn2 = BatchNormalization()(fc2)
  relu2 = ReLU()(bn2)
  d2 = Dropout(0.25)(relu2)
  #  layer3
  fc3 = Dense(32, activation='relu', use_bias=True, kernel_regularizer=L1L2(l1=0.1, l2=0.5))(d2)
  bn3 = BatchNormalization()(fc3)
  relu3 = ReLU()(bn3)
  d3 = Dropout(0.25)(relu3)

  
  # # layer4
  # fc4 = Dense(32, activation='linear', use_bias=True, kernel_regularizer=L1L2(l1=0.1, l2=1))(relu3)
  # bn4 = BatchNormalization()(fc4)
  # d4 = Dropout(0.2)(bn4)
  # relu4 = ReLU()(bn4)
  # softmax
  softmax = Dense(6, activation='softmax', use_bias=True)(d3)



  model_1 = Model(inputs=x, outputs=softmax)
  return model_1

In [ ]:
model_1 = createModel()

In [ ]:
model_1.summary()

# **Test Model1 ==================================**

In [ ]:
model_1.compile(optimizer=Adam(learning_rate=0.0001,)
,loss=SparseCategoricalCrossentropy()
           )# ,metrics=["accuracy"]


In [ ]:
model_1.fit(X_train, y_train, batch_size=64, epochs=100)

In [ ]:
model_1.save('models/v14')
# model = keras.models.load_model('models/v1')

In [ ]:
modemodel_1l = tf.keras.models.load_model('models/v14')

Ptrain = model_1.predict(X_train)
s1 = np.sum(np.argmax(Ptrain, axis=1) == y_train[:,0])/Ptrain.shape[0]
s1

In [ ]:
Ptest = model_1.predict(X_test)
s2 = np.sum(np.argmax(Ptest, axis=1) == y_test[:,0])/Ptest.shape[0]
s2

In [ ]:
model = tf.keras.models.load_model('models/v13')
tests = model.predict(TestImgs)
tests = np.argmax(tests, axis=1)

In [ ]:

col1 = pd.DataFrame(testID,columns=["image_name"]).reset_index(drop=True)
col2 = pd.DataFrame(tests,columns=["label"]).reset_index(drop=True)
sub = pd.concat([col1,col2],axis=1)#.reset_index(drop=True)
sub.to_csv("./submission.csv",index=False)

# **Model2 ==================================**  

In [ ]:
model_2 = Sequential([
    tf.keras.Input(shape=(128,128,3),),

    Conv2D( 32, 5, activation='relu', padding="same"),
    MaxPooling2D(pool_size=8, strides=8, padding='valid'),
    Conv2D( 64, 3, activation='relu', padding="same"),  
    MaxPooling2D(pool_size=4, strides=4, padding='valid'),

    Flatten(),

    Dense(256, activation='relu', kernel_regularizer=L1L2(l1=0.05, l2=0.5),),
    Dropout(0.25),

    Dense(64, activation='relu', kernel_regularizer=L1L2(l1=0.01, l2=0.1),),
    Dropout(0.25),

    Dense(32, activation='relu', kernel_regularizer=L1L2(l1=0.01, l2=0.1),),
    Dropout(0.25),
    
    Dense(6, activation='softmax')
])
    

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(optimizer=Adam(learning_rate=0.01,)
,loss=SparseCategoricalCrossentropy()
           ,metrics=["accuracy"] )#,metrics=["accuracy"]

In [ ]:
model_2.fit(X_train, y_train, batch_size=64, epochs=50)

# **Test Model2 ==================================**

In [ ]:
model_2.save('models/v3_2')
model = tf.keras.models.load_model('models/v3_2')


In [ ]:
Ptest = model.predict(X_train)
s1 = np.sum(np.argmax(Ptest, axis=1) == y_train[:,0])/Ptest.shape[0]

s1

In [ ]:
Ptest = model.predict(X_test)
s2 = np.sum(np.argmax(Ptest, axis=1) == y_test[:,0])/Ptest.shape[0]

s2

In [ ]:
model_2.save('models/v13')
tests = model.predict(TestImgs)
tests = np.argmax(tests, axis=1)
col1 = pd.DataFrame(testID,columns=["image_name"]).reset_index(drop=True)
col2 = pd.DataFrame(tests,columns=["label"]).reset_index(drop=True)
sub = pd.concat([col1,col2],axis=1)#.reset_index(drop=True)
sub.to_csv("./submission.csv",index=False)

# ===========================================================

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(192,192,3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=3,strides=2))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=3,strides=2))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=3,strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=5,strides=4))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=5,strides=4))


model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=6, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001,)
,loss=SparseCategoricalCrossentropy()
           ,metrics=["accuracy"] )#,metrics=["accuracy"]

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=50)

In [ ]:
model_2.save('models/v1')
model = tf.keras.models.load_model('models/v1')

In [ ]:
Ptest = model.predict(X_train)
s1 = np.sum(np.argmax(Ptest, axis=1) == y_train[:,0])/Ptest.shape[0]
del Ptest
del X_train
s1

In [ ]:
Ptest = model.predict(X_test)
s2 = np.sum(np.argmax(Ptest, axis=1) == y_test[:,0])/Ptest.shape[0]
del Ptest
del X_test
s2